In [ ]:
import numpy as np

class PropertyInvestment:
    def __init__(self, purchase_price, down_payment_percent, mortgage_rate, mortgage_term, appreciation_rate,
                 monthly_rent, rent_growth_rate, property_tax_per_100, maintenance_rate, management_fee_rate, 
                 insurance_rate, years_to_hold, closing_costs, selling_cost_rate, vacancy_rate, income_tax_rate,
                 capital_gains_tax_rate, depreciation_period, inflation_rate, property_tax_growth_rate,
                 mortgage_insurance_rate, mortgage_insurance_threshold):
        
        # Initial investment details
        self.purchase_price = purchase_price
        self.down_payment_percent = down_payment_percent
        self.down_payment = purchase_price * down_payment_percent
        self.loan_amount = purchase_price - self.down_payment
        self.mortgage_rate = mortgage_rate
        self.mortgage_term = mortgage_term
        self.appreciation_rate = appreciation_rate
        
        # Income and expenses
        self.monthly_rent = monthly_rent
        self.rent_growth_rate = rent_growth_rate
        self.property_tax_rate = property_tax_per_100 / 100  # Adjusted property tax rate
        self.maintenance_rate = maintenance_rate
        self.management_fee_rate = management_fee_rate
        self.insurance_rate = insurance_rate
        self.years_to_hold = years_to_hold
        self.closing_costs = closing_costs
        self.selling_cost_rate = selling_cost_rate
        self.vacancy_rate = vacancy_rate
        self.income_tax_rate = income_tax_rate
        self.capital_gains_tax_rate = capital_gains_tax_rate
        self.depreciation_period = depreciation_period
        self.inflation_rate = inflation_rate
        self.property_tax_growth_rate = property_tax_growth_rate
        
        # Mortgage insurance parameters
        self.mortgage_insurance_rate = mortgage_insurance_rate
        self.mortgage_insurance_threshold = mortgage_insurance_threshold
        
        # Depreciation calculation
        self.depreciable_basis = self.purchase_price - self.land_value()
        self.depreciation_expense = self.depreciable_basis / self.depreciation_period
        
        # Monthly mortgage payment calculation
        self.monthly_mortgage_payment = self.calculate_monthly_mortgage_payment()
        
        # Initial Investment
        self.initial_investment = self.down_payment + self.closing_costs
        
    def land_value(self):
        """Assume land is 20% of the property value (can be adjusted)."""
        return self.purchase_price * 0.20
    
    def calculate_mortgage_insurance(self):
        """Calculate monthly mortgage insurance premium if down payment is less than the threshold."""
        if self.down_payment_percent < self.mortgage_insurance_threshold:
            # Annual mortgage insurance premium
            annual_mip = self.loan_amount * self.mortgage_insurance_rate
            monthly_mip = annual_mip / 12
        else:
            monthly_mip = 0
        return monthly_mip
    
    def calculate_monthly_mortgage_payment(self):
        """Calculate monthly mortgage payment using loan amortization formula."""
        rate = self.mortgage_rate / 12
        n = self.mortgage_term * 12
        payment = self.loan_amount * rate * (1 + rate)**n / ((1 + rate)**n - 1)
        return payment
    
    def annual_cash_flow(self, year):
        """Calculate annual cash flow for a given year."""
        
        # Adjust property value for appreciation
        property_value = self.purchase_price * (1 + self.appreciation_rate) ** year
        
        # Adjust monthly rent for growth
        annual_rent = self.monthly_rent * (1 + self.rent_growth_rate) ** (year - 1) * 12
        
        # Adjust for vacancy
        annual_rent = annual_rent * (1 - self.vacancy_rate)
        
        # Property tax, adjusted for growth
        property_tax = property_value * self.property_tax_rate * (1 + self.property_tax_growth_rate) ** (year - 1)
        
        # Maintenance cost
        maintenance_cost = property_value * self.maintenance_rate
        
        # Management fee
        management_fee = annual_rent * self.management_fee_rate
        
        # Insurance cost, adjusted for property value
        insurance_cost = property_value * self.insurance_rate
        
        # Mortgage payment (annual)
        mortgage_payment = self.monthly_mortgage_payment * 12
        
        # Mortgage insurance premium (annual)
        monthly_mip = self.calculate_mortgage_insurance()
        annual_mip = monthly_mip * 12
        
        # Depreciation expense
        depreciation_expense = self.depreciation_expense
        
        # Taxable income
        taxable_income = annual_rent - (property_tax + maintenance_cost + management_fee + insurance_cost + mortgage_payment + annual_mip + depreciation_expense)
        
        # Adjust for negative taxable income (can't have negative tax)
        taxable_income_for_tax = max(taxable_income, 0)
        
        # Income tax
        income_tax = taxable_income_for_tax * self.income_tax_rate
        
        # Annual cash flow after taxes
        cash_flow = (annual_rent - (mortgage_payment + annual_mip + property_tax + maintenance_cost + management_fee + insurance_cost + income_tax))
        
        # Adjust cash flow for inflation (present value)
        cash_flow_pv = cash_flow / ((1 + self.inflation_rate) ** year)
        
        return cash_flow_pv
    
    def total_return(self):
        """Calculate total return at the end of the holding period."""
        
        # Calculate present value of cash flows over holding period
        cash_flows = [self.annual_cash_flow(year) for year in range(1, self.years_to_hold + 1)]
        total_cash_flow = sum(cash_flows)
        
        # Final property value after appreciation
        final_property_value = self.purchase_price * (1 + self.appreciation_rate) ** self.years_to_hold
        
        # Selling costs
        selling_costs = final_property_value * self.selling_cost_rate
        
        # Net sale proceeds after selling costs
        net_sale_proceeds = final_property_value - selling_costs
        
        # Outstanding loan balance after years_to_hold
        n_payments_made = self.years_to_hold * 12
        rate = self.mortgage_rate / 12
        n = self.mortgage_term * 12
        remaining_balance = self.loan_amount * ((1 + rate) ** n - (1 + rate) ** n_payments_made) / ((1 + rate) ** n - 1)
        
        # Gain on sale
        adjusted_basis = self.purchase_price + self.closing_costs - (self.depreciation_expense * self.years_to_hold)
        gain_on_sale = net_sale_proceeds - remaining_balance - adjusted_basis
        
        # Capital gains tax
        capital_gains_tax = gain_on_sale * self.capital_gains_tax_rate if gain_on_sale > 0 else 0
        
        # Net proceeds after paying off loan and taxes
        net_proceeds = net_sale_proceeds - remaining_balance - capital_gains_tax
        
        # Total return (cash flows + net proceeds - initial investment)
        total_return = total_cash_flow + net_proceeds - self.initial_investment
        
        return total_return, net_proceeds
    
    def calculate_cagr(self):
        """Calculate the Compound Annual Growth Rate (CAGR) of the investment."""
        beginning_value = self.initial_investment  # Initial investment includes down payment and closing costs
        ending_value = self.total_return()[0] + beginning_value  # Total return + initial investment
        years = self.years_to_hold

        # Apply the CAGR formula
        cagr = (ending_value / beginning_value) ** (1 / years) - 1
        return cagr

    def summary(self):
        """Print a detailed summary of the investment."""
        total_return, net_proceeds = self.total_return()
        cagr = self.calculate_cagr()

        print("\nInvestment Summary")
        print("------------------")
        print(f"Total Return (After Taxes): ${total_return:,.2f}")
        print(f"Net Proceeds from Sale (After Taxes): ${net_proceeds:,.2f}")
        print(f"Average Annual Return (CAGR): {cagr * 100:.2f}%")

        print("\nOutflows")
        print("--------")
        print(f"Purchase Price: ${self.purchase_price:,.2f}")
        print(f"Down Payment ({self.down_payment_percent * 100:.2f}%): ${self.down_payment:,.2f}")
        print(f"Closing Costs: ${self.closing_costs:,.2f}")
        print(f"Total Initial Investment: ${self.initial_investment:,.2f}")

        print("\nInflows")
        print("-------")
        total_cash_flow = sum([self.annual_cash_flow(year) for year in range(1, self.years_to_hold +1)])
        print(f"Total Cash Flow (PV): ${total_cash_flow:,.2f}")
        print(f"Net Proceeds from Sale (After Taxes): ${net_proceeds:,.2f}")

        print("\nAnnual Cash Flows (After Taxes and Inflation Adjusted)")
        print("------------------------------------------------------")
        for year in range(1, self.years_to_hold + 1):
            cash_flow = self.annual_cash_flow(year)
            print(f"Year {year}: ${cash_flow:,.2f}")

        print("\nProperty Value Over Time")
        print("------------------------")
        for year in range(1, self.years_to_hold + 1):
            property_value = self.purchase_price * (1 + self.appreciation_rate) ** year
            print(f"Year {year}: ${property_value:,.2f}")

# Updated parameters with mortgage insurance considerations
purchase_price = 160000
down_payment_percent = 0.10  # Updated to 5% to trigger mortgage insurance
mortgage_rate = 0.0654
mortgage_term = 30
appreciation_rate = 0.03  # 3% annual appreciation
monthly_rent = 1500
rent_growth_rate = 0.02  # 2% annual growth
property_tax_per_100 = 0.7544
maintenance_rate = 0.01 # 1% of property value
management_fee_rate = 0.12  # 12%
insurance_rate = 0.005  # 0.5% of property value
years_to_hold = 5
closing_costs = purchase_price * 0.03  # Assume 3% closing costs
selling_cost_rate = 0.06  # 6% selling costs (agent commissions, etc.)
vacancy_rate = 0.05  # 5% vacancy rate
income_tax_rate = 0.25  # 25% income tax rate
capital_gains_tax_rate = 0.15  # 15% capital gains tax rate
depreciation_period = 27.5  # Residential real estate depreciation period
inflation_rate = 0.02  # 2% annual inflation
property_tax_growth_rate = 0.02  # Property tax rate growth per year
mortgage_insurance_rate = 0.0085  # 0.85% annual mortgage insurance premium (typical for FHA loans)
mortgage_insurance_threshold = 0.10  # Mortgage insurance required if down payment is less than 10%

print("Property Investment Analysis")
print("----------------------------")
print(f"Purchase Price: ${purchase_price:,.2f}")
print(f"Down Payment ({down_payment_percent * 100:.2f}%): ${purchase_price * down_payment_percent:,.2f}")
print(f"Mortgage Rate: {mortgage_rate * 100:.2f}%")
print(f"Mortgage Term: {mortgage_term} years")
print(f"Appreciation Rate: {appreciation_rate * 100:.2f}%")
print(f"Monthly Rent: ${monthly_rent:,.2f}")
print(f"Rent Growth Rate: {rent_growth_rate * 100:.2f}%")
print(f"Vacancy Rate: {vacancy_rate * 100:.2f}%")
print(f"Property Tax Rate: ${property_tax_per_100:.4f} per $100 valuation")
print(f"Property Tax Growth Rate: {property_tax_growth_rate * 100:.2f}%")
print(f"Maintenance Rate: {maintenance_rate * 100:.2f}%")
print(f"Management Fee Rate: {management_fee_rate * 100:.2f}%")
print(f"Insurance Rate: {insurance_rate * 100:.2f}%")
print(f"Closing Costs: ${closing_costs:,.2f}")
print(f"Selling Cost Rate: {selling_cost_rate * 100:.2f}%")
print(f"Income Tax Rate: {income_tax_rate * 100:.2f}%")
print(f"Capital Gains Tax Rate: {capital_gains_tax_rate * 100:.2f}%")
print(f"Depreciation Period: {depreciation_period} years")
print(f"Inflation Rate: {inflation_rate * 100:.2f}%")
print(f"Years to Hold: {years_to_hold}")
print(f"Mortgage Insurance Rate: {mortgage_insurance_rate * 100:.2f}%")
print(f"Mortgage Insurance Threshold: {mortgage_insurance_threshold * 100:.2f}%")

property_investment = PropertyInvestment(
    purchase_price=purchase_price,             # Property purchase price
    down_payment_percent=down_payment_percent, # Down payment percentage
    mortgage_rate=mortgage_rate,               # Annual mortgage interest rate
    mortgage_term=mortgage_term,               # Mortgage term in years
    appreciation_rate=appreciation_rate,       # Annual appreciation rate
    monthly_rent=monthly_rent,                 # Initial monthly rental income
    rent_growth_rate=rent_growth_rate,         # Annual rental growth rate
    property_tax_per_100=property_tax_per_100, # Property tax rate per $100 valuation
    maintenance_rate=maintenance_rate,         # Annual maintenance rate
    management_fee_rate=management_fee_rate,   # Property management fee rate
    insurance_rate=insurance_rate,             # Annual insurance rate
    years_to_hold=years_to_hold,               # Number of years to hold the property
    closing_costs=closing_costs,               # Initial closing costs
    selling_cost_rate=selling_cost_rate,       # Selling cost rate
    vacancy_rate=vacancy_rate,                 # Vacancy rate
    income_tax_rate=income_tax_rate,           # Income tax rate on rental income
    capital_gains_tax_rate=capital_gains_tax_rate,  # Capital gains tax rate
    depreciation_period=depreciation_period,   # Depreciation period
    inflation_rate=inflation_rate,             # Inflation rate
    property_tax_growth_rate=property_tax_growth_rate,  # Property tax rate growth per year
    mortgage_insurance_rate=mortgage_insurance_rate,  # Mortgage insurance rate
    mortgage_insurance_threshold=mortgage_insurance_threshold  # Down payment threshold for mortgage insurance
)

# Generate the investment summary
property_investment.summary()


Property Investment Analysis
----------------------------
Purchase Price: $160,000.00
Down Payment (10.00%): $16,000.00
Mortgage Rate: 6.54%
Mortgage Term: 30 years
Appreciation Rate: 3.00%
Monthly Rent: $1,500.00
Rent Growth Rate: 2.00%
Vacancy Rate: 5.00%
Property Tax Rate: $0.7544 per $100 valuation
Property Tax Growth Rate: 2.00%
Maintenance Rate: 1.00%
Management Fee Rate: 8.00%
Insurance Rate: 0.50%
Closing Costs: $4,800.00
Selling Cost Rate: 6.00%
Income Tax Rate: 25.00%
Capital Gains Tax Rate: 15.00%
Depreciation Period: 27.5 years
Inflation Rate: 2.00%
Years to Hold: 2
Mortgage Insurance Rate: 0.85%
Mortgage Insurance Threshold: 10.00%

Investment Summary
------------------
Total Return (After Taxes): $269.31
Net Proceeds from Sale (After Taxes): $18,861.71
Average Annual Return (CAGR): 0.65%

Outflows
--------
Purchase Price: $160,000.00
Down Payment (10.00%): $16,000.00
Closing Costs: $4,800.00
Total Initial Investment: $20,800.00

Inflows
-------
Total Cash Flow (PV): $2,20